# Project 2: Span Identification with Sequence Labeling Models
## CS4740/5740 Fall 2020

### Project Submission Due: Oct 23rd
Please submit **pdf file** of this notebook on **Gradescope**, and **ipynb** on **CMS**. For instructions on generating pdf and ipynb files, please refer to project 1 instructions.



Names: Yuanhong Cao, Zeyu Chen

Netids: yc2668, zc436

Don't forget to share your newly copied notebook with your partner!


**Reminder: both of you can't work in this notebook at the same time from different computers/browser windows because of sync issues. We even suggest to close the tab with this notebook when you are not working on it so your partner doesn't get sync issues.**


### Q0: Individual Member Contribution

Briefly explain the contribution of individual group members here. Report if working loads are unfairly distributed.

Zeyu mainly focused on Model 1 and Yuanhong mainly focused on Model 2 and we finished together for all the questions. We cooperated well for this project.

# Overview

---

In this project, you will implement a model that identifies relevant information in a text and tags it with the appropriate label. Particularly, the task of this project is **Propaganda Identification**. The given dataset contains (manual) annotations indicating fragments of text that exhibit one of a set of well-known [propaganda techniques](https://propaganda.qcri.org/annotations/definitions.html). Your task is to develop NLP models to identify these propagandistic spans of text automatically. We will treat this as a **sequence-tagging task**: for each token in the input text, assign a label $y\in\{0,1\}$, such that *1 represents propagandistic text* and *0 represents non-propaganda*.   (A description of the original task formulation is [here](https://propaganda.qcri.org/ptc/).  We are working on a modified version of their "span identification" task.)

For this project, you will implement two sequence labeling approaches:

- Model 1 : a Hidden Markov Model (HMM)
- Model 2 : a Maximum Entropy Markov Model (MEMM), which is an adaptation of an HMM in which a Logistic Regression classifier (also known as a MaxEnt classifier) is used to obtain the lexical generation probabilities (i.e., the observation/emission probability matrix, so "observations" == "emissions" == "lexical generations").  Feature engineering is strongly suggested. (Papers from the [Workshops on Figurative Language Processing](https://sites.google.com/view/figlang2020/) can provide good insights on feature selection for this task.) You can also refer to the J&M book. 

Implementation of the Viterbi algorithm (for finding the most likely tag sequence to assign to an input text) is required for both parts, so make sure that you understand it ASAP.

You will implement and train two sequence tagging models, generate your predictions for the provided test set, and submit them to **Kaggle**. Please enter all code and answer the questions of this colab notebook.

**Jurafsky & Martin reading on HMMs and MEMMs can be found in Ch. 8.3–8.5.** The code you write can be added anywhere in the document, but we implore you to keep it readable. You will be asked to describe and motivate your models in parts of the document. You will be graded on both the code and text you write; see grading details at the end of the document.


# Notes

---

1. Please read through the entire notebook before you start coding. That might inform your code structure.
2. Grading breakdown is found at the end; please consult it.
3. Google colab does **not** provide good synchronization; we do not recommend multiple people to work on the same notebook at the same time.
4. The project is somewhat open ended. ("But that's a good thing.  Really. It's more fun that way", says Claire and Esin.) We will ask you to implement some model, but precise data structures and so on can be chosen by you. However, to integrate with Kaggle, you will need to submit Kaggle predictions using our tokenization code.  As a result, **it is probably easiest if you use our tokenization code for the entire project**.
5. You will be asked to fill in your code at various points of the document. You will also be asked to answer questions that analyze your results and motivate your implementation. These questions are named Q1-Q8. You may create additional cells to insert code, text and images as necessary.
6. Kaggle is not able to calculate *span-level* P/R/F1 measures, which is the standard way to evaluate this type of sequence-tagging task. And we don't actually care so much about the token-level tagging accuracy, which Kaggle **can** calculate.  In particular, there are many fewer propaganda tokens than non-propaganda ones, so always guessing "non-propaganda" would produce a very high accuracy.  So we are compromising by using token-level **weighted accuracy**.  Here is how it works:

A **weighted accuracy** metric that favors finding propagandistic tokens over non-propagandistic ones. The weights for both classes are the inverse of their frequencies.  

``` 
frac_propaganda = num_propaganda/num_labels   [in the answer key]
weight_propaganda = 1/frac_propaganda  
weight_non_propaganda = 1/(1-frac_propaganda)

weighted_accuracy = 
   ((weight_propaganda * # propaganda correct) 
                      +
   (weight_non_propaganda * # non_propaganda correct)) / 
   
   ((weight_propaganda * num_propaganda) 
                      +
   (weight_non_propaganda * num_non_propaganda))
```  
This is also known as the **macro average**, i.e., the average of the accuracy for each label type.


# Task and dataset

---

1. Obtain the data from Kaggle at https://www.kaggle.com/t/8a8030baefcc4d91b715f114353dba38.
2. Unzip the data. Put it into your google drive, and mount it on colab as per below:

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)



train_path = os.path.join(os.getcwd(), "drive", "My Drive", "CS 4740", "Project 2", "Dataset", "train") # replace based on your Google drive organization
test_path = os.path.join(os.getcwd(), "drive", "My Drive", "CS 4740", "Project 2", "Dataset", "test") # replace based on your Google drive organization

"""
  To Check if the data load correctly, and how many files in train and test folders.
"""
print(len(os.listdir(train_path)))
print(os.listdir(train_path)[:5])
print(len(os.listdir(test_path)))
print(os.listdir(test_path)[:5])

Mounted at /content/drive


FileNotFoundError: ignored

3. The *train* directory contains *article{XXX}.txt* files which are plaintext documents and also *label* files such as *article{XXX}.task-si.labels*. These label files correspond to the byte-span offests of each segment of propaganda in the associated article. (The tokenizer that we describe just below converts the byte-span representation of propagandistic text spans  into the token-level gold-standard labels that your sequence-tagging models require for training.) The test directory *only* contains articles; you will use your models to detect the propagandistic spans within them.  

4. We provide a tokenizer for these documents. You **must** use this tokenizer as the labels that Kaggle expects are based upon this tokenization. The code below tokenizes each document and generates the appropriate token-level labels consistent with the associated *labels* file. (This is so that you do not need to perform any byte-level text processing.  In particular, the tokenizer  merges nested or overlapping propagandistic text spans from the article into a single segment. You really shouldn't have to look at the *lables* files at all.) The code uses python type annotations; these indicate the type of arguments the functions take.

5. Documents are represented as a list of strings, each being a token. Labels are represented as a list of integers in {0,1}, 1 corresponding to a propagandistic token and 0 to not propaganda.



In [ ]:
import os
from typing import List, Tuple

from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

def read_txt(fname):
  with open(fname) as open_article:
    lines = open_article.read()
  return lines

def read_labels(labels : str) -> List[Tuple[int, int]]:
	"processing of labels file"
	labels = labels.split("\n")[:-1]
	labels = [tuple(map(int, l.split("\t")[1:])) for l in labels]
	return labels

def sort_and_merge_labels(labels : List[Tuple[int, int]]) -> List[Tuple[int, int]]:
  "sort labels, necessary for later splitting"
  if len(labels) == 0:
    return labels
  labels = list(sorted(labels, key = lambda t: t[0]))
  # merge
  curr = labels[0]
  merged = []
  for l in labels[1:]:
      # if distinct, add
      if l[0] > curr[1]:
        merged.append(curr)
        curr = l
      # else merge
      else:
        curr = (curr[0], max(curr[1], l[1]))
  merged.append(curr)
  return merged

def split_with_labels(labels : List[Tuple[int, int]], article : str) -> Tuple[List[str], List[int]]:
  "split text into segments based upon labels"
  if len(labels) == 0:
    return [article], [0]
  segments = []
  binary_class = []
  start = 0
  for l_start, l_end in labels:
    std_seg = article[start:l_start]
    prop_seg = article[l_start:l_end]
    segments.append(std_seg)
    binary_class.append(0)
    segments.append(prop_seg)
    binary_class.append(1)
    start = l_end
  last_seg = article[start:]
  segments.append(last_seg)
  binary_class.append(0)
  return segments, binary_class

def remove_newline_fix_punc_seg(segments):
  " preprocessing necessry for tokenization to be consistent"
  segments = [s.replace("\n", " ").replace(".", " .") for s in segments]
  return segments

def remove_newline_fix_punc_art(article):
  " preprocessing necessry for tokenization to be consistent"
  article = article.replace("\n", " ").replace(".", " .")
  return article

def get_toks(input):
  output = []
  for toks in [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(input)]:
    output += toks
  return output

# This is the function you may need to call
def tokenize_article(article_file):
  "calls all functions above and perform sanity checks"
  article = read_txt(article_file)
  article = remove_newline_fix_punc_art(article)
  art_toks = get_toks(article)
  return art_toks

# This is the function you may need to call
def master_tokenizer(article_file, labels_file):
  "calls all functions above and perform sanity checks"
	# read and get labels
  article = read_txt(article_file)
  labels = read_txt(labels_file)
  labels = read_labels(labels)
  labels = sort_and_merge_labels(labels)
  segments, binary_class = split_with_labels(labels, article)
  article = remove_newline_fix_punc_art(article)
  segments = remove_newline_fix_punc_seg(segments)
  # sanity check
  reconstructed = ""
  for seg, lab in zip(segments, binary_class):
    reconstructed += seg
  assert reconstructed == article
	# tokenize
  seg_toks = []
  new_labels = []
  for seg, label in zip(segments, binary_class):
    new_toks = get_toks(seg)
    seg_toks += new_toks
    new_labels += [label for _ in range(len(new_toks))]
	# sanity check
  art_toks = get_toks(article)
  sanity = True
  if len(art_toks) != len(seg_toks):
    sanity = False
  for i, (at, st, lab) in enumerate(zip(art_toks, seg_toks, new_labels)):
    if at != st:
      sanity = False
      break
  return seg_toks, new_labels, sanity




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 6. Execute the commands below to visualize the tokenization:

In [ ]:
article_file = "article698018235.txt"
labels_file = "article698018235.task-si.labels"
article_file = os.path.join(train_path, article_file)
labels_file = os.path.join(train_path, labels_file)
tokens, labels, _ = master_tokenizer(article_file, labels_file)

print(len(tokens), len(labels), tokens[:10])

['u', '.s', '.', 'judge', 'frees', 'indonesian', 'immigrant', 'held', 'by', 'trump']
['u', '.s', '.', 'judge', 'frees', 'indonesian', 'immigrant', 'held', 'by', 'trump']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


7.  Provide some quantitative data exploration. Assess dataset size, documents lengths and class inbalance. Give some examples of sentences containing propaganda techniques.

In [ ]:
from statistics import mean 
print(len(os.listdir(train_path)))
print(len(os.listdir(test_path)))

token_files = []
for f in os.listdir(train_path):
  if f.endswith(".txt"):
    token_files.append(f)
label_files = [f.replace('.txt', '.task-si.labels') for f in token_files]

len_lst = []
class_lst = []
for i in range(len(token_files)):
  ob_token = os.path.join(train_path, token_files[i])
  ob_label = os.path.join(train_path, label_files[i])
  ob_t, ob_l, _ = master_tokenizer(ob_token, ob_label)
  len_lst.append(len(ob_t))
  class_lst.append(sum(ob_l))
print(mean(len_lst), max(len_lst), min(len_lst))
print(mean(class_lst), max(class_lst), min(class_lst))

520
47
966.4423076923077 8305 120
101.91153846153846 1604 0


### Q1: Initial data observations
What are your initial observations of the dataset after you explore the dataset?

**Answer:** There are 520 files in total (260 txt files and 260 label files) in the training set and 47 files in total in the testing set. In the training set, the average document length is cloase to 966 with maximum 8305 and minimum 120. The average number of propagandistic tokens is close to 120 with maximum 1604 and minimum 0.  

# Model 1: HMM Implementation

---

In this section, you will implement a HMM model for this task. We expect:


1. An implementation of the **Viterbi algorithm** that can be used to infer token-level labels --- propaganda or not propaganda --- for an input document.   This process is commonly referred to as **decoding**.
2. Code for counting and smoothing of labels and words as necessary to support the HMM decoding. (This is pretty much what you already know how to do from project 1.)


The tokenization of documents can be performed with the code we provide above. We suggest you calculate probabilities in a log form.  Bigram Viterbi is $ \mathcal{O}(sm^2)$ where s is the length of the sentence and m is the number of tags. Your implementation should have similar efficiency.

Code of Academic Integrity:  We encourage collaboration regarding ideas, etc. However, please **do not copy code from online or share code with other students**. We will be running programmes to detect plagiarism.

In [ ]:
import math
import collections
nltk.download('all')
from nltk.book import *

"""Unknown handling"""
def unknown_handling(lst):
  fdist = FreqDist(lst)
  top70 = [word for (word, _) in fdist.most_common(int(len(set(lst)) * 0.7))]
  mapping = collections.defaultdict(lambda: 'UNK')
  for w in top70:
    mapping[w] = w
  filtered = [mapping[w] for w in lst]
  return filtered

"""Merge two list as a list of tuples"""
def merge(list1, list2): 
  merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))] 
  return merged_list

"""
Function emission_build: computes the emission dictionary of a list
lst: a list of tags that we want to compute
prop_coun: count of tag 1 in the lst
nonprop_coun: count of tag 0 in the lst
Return: a dictionary that stores the log(probability) between word and tag
"""
def emission_build(lst, prop_count, nonprop_count):
  result = collections.defaultdict(lambda: 0)
  for item in lst:
    result[item] += 1
  for item in result:
    if item[1] == 0:
      result[item] = math.log(result[item] / nonprop_count)
    else:
      result[item] = math.log(result[item] / prop_count)
  return result

"""
Function transition_build: computes the transition dictionary of a list
lst: a list of words that we want to compute
k: smoothed number we want to add
Return: a dictionary that stores the log(probability) between tags
"""
def transition_build(lst, k):
  uni_dic = collections.defaultdict(lambda: 0)
  for w in lst:
    uni_dic[w] += 1
  bilst = list(bigrams(lst))
  bi_dic = collections.defaultdict(lambda: 0)
  for w in bilst:
    bi_dic[w] += 1
  unique = len(uni_dic)
  result = bi_dic
  for w in bi_dic:
    numerator = bi_dic[w] + k
    denominator = uni_dic[w[0]] + unique * k
    result[w] = math.log(numerator / denominator)
  return result

"""
Function viterbi: computes the probable tag path of a sequence
c: number of lexical catagories, which is 2
lst: a list of words that we want to predict
emission: the emission dictionary storing log(probability)
transition: the transition dictionary storing log(probability)
Return: a list that stores the result
"""
def viterbi(c, lst, percent_prop, emission, transition):
  n = len(lst)
  score = [[0 for i in range(n)] for j in range(c)]
  bptr = [[0 for i in range(n)] for j in range(c)]
  for i in range(0, c):
    if i == 0:
      if (lst[0], i) in emission:
        score[i][0] = math.log(1 - percent_prop) + emission[(lst[0], i)]
      else:
        score[i][0] = math.log(1 - percent_prop) + emission[('UNK', i)]
    else:
      if (lst[0], i) in emission:
        score[i][0] = math.log(percent_prop) + emission[(lst[0], i)]
      else:
        score[i][0] = math.log(percent_prop) + emission[('UNK', i)]
    bptr[i][0] = 0
  for t in range(1, n):
    for i in range(0, c):
      tmp = (score[0][t - 1] + transition[(0, i)], score[1][t - 1] + transition[(1, i)])
      if (lst[t], i) in emission:
        score[i][t] = max(tmp) + emission[(lst[t], i)]
      else:
        score[i][t] = max(tmp) + emission[('UNK', i)]
      bptr[i][t] = tmp.index(max(tmp))
  T = [0 for i in range(n)]
  tmp = (score[0][n - 1], score[1][n - 1])
  T[n - 1] = tmp.index(max(tmp))
  for i in range(n - 2, -1, -1):
    T[i] = bptr[T[i + 1]][i + 1]
  return T

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

## Validation Step

1. Create a validation set from the given dataset, i.e. a subset of (~10%) the training dataset that you only use for evaluating the models, not for training.  (You can think of the validation set as a sample test set.)
2. Train your HMM model on the (remainder of the) training set and evaluate it on the validation set. Report **weighted accuracy**, which is explained in the *Notes* section above.

Please also take a look into your misclassified cases, as we will be performing error analysis in the *Evaluation* section. We expect smoothing, unknown word handling and correct emission (i.e., lexical generaion) probabilities. 

In the *Kaggle Submission* section, there is code provided for generating the output file in the form required for Kaggle.  If you find it useful for computing weighted accuracy, you can use it here as well.

In [ ]:
"""
  Split into training set and validation set
  We use the whole training set for training, 
  and 10% of them for validation.
"""

import random

random.seed(1)
random.shuffle(token_files)
print(token_files)
validation_tokens = token_files[int(len(token_files)*0.9):]

validation_labels = [f.replace('.txt', '.task-si.labels') for f in validation_tokens]

['article699291100.txt', 'article729348908.txt', 'article724791253.txt', 'article771546417.txt', 'article999000124.txt', 'article700551604.txt', 'article722512241.txt', 'article999000880.txt', 'article754111899.txt', 'article727869189.txt', 'article729581752.txt', 'article735090638.txt', 'article111111132.txt', 'article111111133.txt', 'article701553469.txt', 'article758472954.txt', 'article763440871.txt', 'article770938827.txt', 'article786344683.txt', 'article999001619.txt', 'article758385628.txt', 'article999000312.txt', 'article771879020.txt', 'article762200179.txt', 'article710100700.txt', 'article769682854.txt', 'article731762731.txt', 'article780787440.txt', 'article788173482.txt', 'article755393220.txt', 'article706501640.txt', 'article774904810.txt', 'article730036755.txt', 'article999000125.txt', 'article769962236.txt', 'article730389374.txt', 'article761610997.txt', 'article758469195.txt', 'article761546223.txt', 'article770945799.txt', 'article999000565.txt', 'article7305598

In [ ]:
"""
  Retrive tokenize-level data for word tokens and labels of both training and validation.
"""
training_tokens_combine = []
training_labels_combine = []
for i in range(len(token_files)):
  article_f = os.path.join(train_path, token_files[i])
  labels_f = os.path.join(train_path, label_files[i])
  tokens2, labels2, _ = master_tokenizer(article_f, labels_f)
  training_tokens_combine = training_tokens_combine + tokens2
  training_labels_combine = training_labels_combine + labels2

validation_tokens_combine = []
validation_labels_combine = []
for i in range(len(validation_tokens)):
  article_f = os.path.join(train_path, validation_tokens[i])
  labels_f = os.path.join(train_path, validation_labels[i])
  tokens2, labels2, _ = master_tokenizer(article_f, labels_f)
  validation_tokens_combine = validation_tokens_combine + tokens2
  validation_labels_combine = validation_labels_combine + labels2

In [ ]:
"""
  Calculate Percent of Propaganda, emission rate, transition rate for Viterbi
  Calculate Viterbi
"""
prop_count = sum(training_labels_combine)
nonprop_count = len(training_labels_combine) - prop_count
percent_prop = prop_count / len(training_labels_combine)
print(prop_count, nonprop_count, percent_prop)

handled_training_tokens_combine = unknown_handling(training_tokens_combine)
token_label = merge(handled_training_tokens_combine, training_labels_combine)
emission = emission_build(token_label, prop_count, nonprop_count)
transition = transition_build(training_labels_combine, 3)
result = viterbi(2, validation_tokens_combine, percent_prop, emission, transition)

19212 234679 0.07567026794963193


In [ ]:
"""
  HMM Weighted Accuracy Calculation
"""
prop_correct = 0
nonprop_correct = 0
for i in range(len(result)):
  if result[i] == validation_labels_combine[i]:
    if result[i] == 0:
      nonprop_correct += 1
    else:
      prop_correct += 1

frac_propaganda = sum(validation_labels_combine)/len(validation_labels_combine)
weight_propaganda = 1 / frac_propaganda
weight_non_propaganda = 1 / (1 - frac_propaganda)
weighted_accuracy = (weight_propaganda * prop_correct 
                     + weight_non_propaganda * nonprop_correct) / (weight_propaganda * sum(validation_labels_combine) 
                     + weight_non_propaganda * (len(validation_labels_combine) - sum(validation_labels_combine)))
print(weighted_accuracy)


0.4932487912674215




### Q2 : Explan your HMM implementations

Q2.1: Explain here how you implemented HMMs (e.g. **which algorithms/data structures** you used). Make clear which parts were implemented from scratch vs. obtained via an existing package. 

**Answer:** There are three parts in HMM: emission matrix, transition matrix and Viterbi function. For the first ftwo matrix, we used structure dictionary to store the log(probability) because the elements are easy to store and get. For the transition function, we used our add_k_bigram function from project 1. For unknown handling, we used our unknown_handling function from project 1. For Viterbi function, we basically followed the pseudocode in the lecture notes. We did not use any existing package when building the model.

Q2.2: Explain and motivate any design choices providing the intuition behind them (e.g. which methods you used for your HMM implementation, why?).

**Answer:** Basically we followed the pseudocode in the lecture notes, so we choose Viterbi method for our HMM model.



### Q3: Results Analysis

Q3.1: Explain here how you evaluated the models. Summarize the performance of your system and any variations that you experimented with on the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis. 

**Answer:** We followed the instruction above and split the 10% of training files as validation files. We used all the training files to train the models and use the validation files to evaluate the models. After unknown word handling and smoothing, the weighted accuracy is close to 50%.




Q3.2: When did the system work well? When did it fail?  Any ideas as to why? How might you improve the system?

**Answer:** There is not much difference among performance. But when we change the percentage of unknown word handling and smoothed value, the results are different slightly. Because changeing the unknown handling and smoothing will change the emission and transition matrix, so it will affect the result. For better performance, we can make unknown word handling percentage relative large and make k relative small.

Q3.3: What is the effect of unknown word handling and smoothing?

**Answer:** For the validation part, unknown word handling and smoothing did not help much. The reason may be that almost all combinations have already appeared in other training files.



# Model 2: MEMM Implementation

---


In this section, you will implement a Maximum Entropy Markov Model (**MEMM**) to perform the same propaganda detection task. Your model should consist of a MaxEnt classifier with Viterbi decoding. 
 
1. We have already performed tokenizations for documents. You can either use a MaxEnt classifier from an existing package or write the MaxEnt code yourself. **Important note:  MaxEnt classifiers are statistically equivalent to multi-class logistic regression, so you can use packages for multi-class LR instead of MaxEnt.**

2. Use the classifier to learn a probability $P(t_i|features)$. You may replace either the lexical generation probability – $P(w_i|t_i)$ – or the transition probability – $P(t_i|t_{i−1})$ – in the HMM with it, or you may replace the entire *lexical generation probability * transition probability*  calculation – $P (w_i|t_i) ∗ P (t_i|t_{i−1)} – $ in the HMM with it. 

3. To train such classifier, you need to pick some feature set. The content of the feature set is up to your choice. You should be trying different feature sets, and evaluate your choices on the validation set. Pick the feature set that performs overall the best according to the F1 measure.

4. Use your own implementation of the **Viterbi algorithm**, which you can modify from the one you developed for the HMM model. You will need the probabilities that you obtain from the MaxEnt classifier. 

5. Remember to use same training and validation split when evaluating the MEMM to have a **fair comparison** with your **HMM model**.


Please also take a look into your misclassified cases, as we will be performing error analysis in *Evaluation* section. 





---
Work flow summary:

![alt text](https://drive.google.com/uc?export=view&id=14VfjW3yDyXLojWM_u0LeJYdDOSLkElBn)




In [ ]:
# 1. Extract Features
"""
  Put All Article files path in one list, as well as All Label files
  Take sure they are in same order and corresppond with each other
"""
allfile = os.listdir(train_path)
alllabelsfile = []
allarticlefile = []
for labelsfile in allfile:
  if labelsfile.endswith('task-si.labels'):
    alllabelsfile.append(labelsfile)
    articlefile = labelsfile.replace("task-si.labels", "txt")
    allarticlefile.append(articlefile)
features = []
alllabels = []

"""
  Extract Features "word" (Single Word Token), 
  "pretag" (the Propaganda tag/label of previous word token), 
  "pos" (Part-Of-Speech tag of current word token);
  Manually set the first word token of each article with 'word': 'first word', 'pretag': 0, 'pos': 'NN'. 
"""
for i in range(0, len(allarticlefile)):
  firstwordfeature = {'word': allarticlefile[0], 'pretag': 0, 'pos': nltk.tag.pos_tag([allarticlefile[0]])[0][1]}
  features.append(firstwordfeature)
  article_file = os.path.join(train_path, allarticlefile[i])
  labels_file = os.path.join(train_path, alllabelsfile[i])
  tokens, labels, _ = master_tokenizer(article_file, labels_file)
  alllabels = alllabels + labels
  for j in range(1, len(tokens)):
    feature = {}
    feature['word'] = tokens[j]
    feature['pretag'] = labels[j-1]
    tag = nltk.tag.pos_tag([tokens[j]])
    feature['pos'] = tag[0][1]
    features.append(feature)


In [ ]:
# 2. Train MaxEnt
import pandas as pd
import numpy as np
import operator
from sklearn.linear_model import LogisticRegression
import nltk.classify.util
from nltk.classify import maxent

"""
  Put features and labels one-to-one correspondence
"""
fd_list = []

for i in range(len(features)):
  fd = []
  fd.append(features[i])
  fd.append(alllabels[i])
  fd_list.append(fd)

"""
  Training MEMM classifier
"""
classifier = maxent.MaxentClassifier.train(fd_list, max_iter=30)


  ==> Training (30 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.105
             2          -0.12572        0.903
             3          -0.08750        0.975
             4          -0.06691        0.979
             5          -0.05612        0.979
             6          -0.04995        0.979
             7          -0.04609        0.979
             8          -0.04351        0.979
             9          -0.04167        0.979
            10          -0.04031        0.981
            11          -0.03926        0.981
            12          -0.03843        0.981
            13          -0.03776        0.981
            14          -0.03721        0.981
            15          -0.03674        0.981
            16          -0.03634        0.981
            17          -0.03600        0.981
            18          -0.03570        0.981
            19          -0.03543        0.981
  

In [ ]:
# 3. To call Viterbi 
"""
  Implement MEMM Viterbi function, 
  Input is the number of class and list of test/validation token,
  Return the Predicted results (list of labels), and Log Score
  Also manually set the first token as 'word': 'first word', 'pretag': 0, 'pos': 'NN'
"""
import math
def memm_viterbi(c, test_tokens):
  results = []
  test_feature = {'word': test_tokens[0], 'pretag': 0, 'pos': nltk.tag.pos_tag([test_tokens[0]])[0][1]}
  if classifier.prob_classify(test_feature).prob(0) > 0.5:
    pretag = 0
    results.append(pretag)
    score = math.log(classifier.prob_classify(test_feature).prob(0))
  else:
    pretag = 1
    results.append(pretag)
    score = math.log(classifier.prob_classify(test_feature).prob(1))
  for i in range(1, len(test_tokens)):
    test_feature = {'word': test_tokens[i], 'pretag': pretag, 'pos': nltk.tag.pos_tag([test_tokens[i]])[0][1]}
    if classifier.prob_classify(test_feature).prob(0) > 0.5:
      pretag = 0
      results.append(pretag)
      score = score + math.log(classifier.prob_classify(test_feature).prob(0))
    else:
      pretag = 1
      results.append(pretag)
      score = score + math.log(classifier.prob_classify(test_feature).prob(1))
  return results, score


In [ ]:
"""
  Use the same Validation Set to call MEMM Viterbi, get the results and score.
"""
results, score = memm_viterbi(2, validation_tokens_combine)
print(score)

-475.8065730367333
26795
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
"""
  Caculate the Weighted Accuracy of MEMM Viterbi Results
"""
prop_correct = 0
nonprop_correct = 0
for i in range(len(results)):
  if results[i] == validation_labels_combine[i]:
    if results[i] == 0:
      nonprop_correct += 1
    else:
      prop_correct += 1
frac_propaganda = sum(validation_labels_combine)/len(validation_labels_combine)
weight_propaganda = 1 / frac_propaganda
weight_non_propaganda = 1/(1-(frac_propaganda))
weighted_accuracy = (weight_propaganda * prop_correct 
                     + weight_non_propaganda * nonprop_correct) / (weight_propaganda * sum(validation_labels_combine) 
                     + weight_non_propaganda * (len(validation_labels_combine) - sum(validation_labels_combine)))

print(weighted_accuracy)

0.5549605489356105


### Q4: Implementation Details

Q4.1: Explain here how you implemented the MEMM (e.g. which algorithms/data structures you used, what features are included). Make clear which parts were implemented from scratch vs. obtained via an existing package. 


**Answer:** For the features of MEMM, we choose three features, Current Word Token("word"), Label/tag of Previous Word Token("pretag"), and Part-of-Speech tag("pos"). Then we paired them with corresponding correct result. We passed them into the MaxentClassifier of nltk for training, therefore we got the classifier for further testing.

Q4.2: What features are considered most important by your MaxEnt Classifier? Why do you think these features make sense? Describe your experiments with feature sets.

**Answer:** I think all of them are important: for "word", we have to know this word to decide whether it is propaganda or nort; for "pretag", the current tag is depend on previous tag as we learned how the MEMM and Viterbi works in class; for "pos", the certain type of pos has higher probability to be propaganda. And when I delete one of them, the result would be worse, so I think they are all important.



### Q5: Results Analysis

Q5.1: Explain here how you evaluated the MEMM model. Summarize the performance of your system and any variations that you experimented with the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis. 

**Answer:** We tried several times on different random validation set. And compare the predicted result set with currect label set to see the different.

Q5.2: An analysis on feature selection for the MEMM is required – e.g. what features **help most**, why? An **error analysis** is required – e.g. what sorts of errors occurred, why? 

**Answer:** I think "pretag" helps most. Without it, the MEMM model will suffer significant False Negative problem.

Q5.3: When did the system work well, when did it fail and any ideas as to why? How might you improve the system?

**Answer:** When I set all my three features, it works well. I might try to use more new features in the future to improve the system.




#Comparing HMMs and MEMMs

---


### Q6:


Compare here your results for your HMM and the MEMM. Which of them performs better? 

**Answer:** From Weighted Accuracy, the MEMM is better.


###Q7: Error Analysis
Q7.1: Do some error analysis. What are error patterns you observed that the HMM makes but the MEMM does not? Try to justify why/why not?

**Answer:** HMM always predicts 0 as 1, which is False Positive. And MEMM does not.

Q7.2: What are error patterns you observed that MEMM makes but the HMM does not? Try to justify what you observe?

**Answer:** MEMM always predicts 1 as 0, which is False Negative. And HMM does not.




# Kaggle Submission 
---

Using the best-performing system from among all of your HMM and MEMM models, generate predictions for the test set, and submit them to Kaggle at https://www.kaggle.com/t/8a8030baefcc4d91b715f114353dba38. Note, you **need** to use our tokenizer as the labels on Kaggle corresponds to these. Below, we provide a script that submits a random guess/all ones or all zeroes in the correct format. Note that you only need to provide a function which takes as input a sequence of tokens, and outputs a sequence of labels (in the form of integers  in {0,1}, 1 corresponding to propaganda). As a scoring metric on Kaggle, we use **weighted accuracy**, described in the *Notes* section towards the beginning of the notebook.


In [ ]:
import random
from typing import Callable


def submission_to_csv(binary_class : List[List[int]], fname : str):
  # writes submission to CSV
  new_binary = []
  for classes in binary_class:
    new_binary += classes
  with open(fname, 'w') as file:
    file.write('id,category\n')
    for i, line in enumerate(new_binary):
      file.write('{},{}\n'.format(i,line))


def random_predictor(article_tokens : List[str]) -> List[int]:
  # random predictor
  output = []
  for _ in range(len(article_tokens)):
    output.append(random.randint(0,1))
  return output

def always_one(article_tokens : List[str]) -> List[int]:
  # random predictor
  output = []
  for _ in range(len(article_tokens)):
    output.append(1)
  return output

def always_zero(article_tokens : List[str]) -> List[int]:
  # random predictor
  output = []
  for _ in range(len(article_tokens)):
    output.append(0)
  return output


def generate_submission(predictor: Callable[[List[str]], List[int]], fname : str, test_path : str):
  # generate a submission with random guesses
  sample_submission = []
  articles = os.listdir(test_path)
  total, fail, targets, test_articles = 0,0, [], []
  for article in sorted(articles):
    article_file = os.path.join(test_path, article)
    art_toks = tokenize_article(article_file)
    curr_article = predictor(art_toks)
    sample_submission.append(curr_article)
  submission_to_csv(sample_submission, fname)


generate_submission(random_predictor, "random_submission.csv", test_path)
generate_submission(always_one, "ones_submission.csv", test_path)
generate_submission(always_zero, "zeros_submission.csv", test_path)


In [ ]:
def our_prediction(article_tokens : List[str]) -> List[int]:
  output = []
  lst, score = memm_viterbi(2, article_tokens)
  for _ in range(len(article_tokens)):
    output.append(lst[_])
  return output
generate_submission(our_prediction,"test.csv",test_path)

---
### Q8: Competition Score

Include here your **team name** and the **screenshot** of your best score from Kaggle.

**Answer:** team name: yc2668_zc436

screenshot of kaggle:
![alt text](https://drive.google.com/uc?export=view&id=1bbUyiAkqOb_0vwba5hLQNLoq7NTg5eqF)


In [1]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [2]:
%%capture
# the red text is a placeholder! Change it to your directory structure!
!cp 'drive/My Drive/CS 4740/Project 2/4740_FA20_p2_yc2668_zc436.ipynb' ./ 

In [3]:
# the red text is a placeholder! Change it to the name of this notebook!
!jupyter nbconvert --to PDF "4740_FA20_p2_yc2668_zc436.ipynb"

[NbConvertApp] WARNING | pattern u'4740_FA20_p2_yc2668_zc436.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--inp

#Appendix
---

### Tentative grading guideline

- (5 pts) Dataset exploration.
- (20 pts) Design and implementation of the HMM model. Specifically, calculations of emission/transition probability, smoothing, handling of unknown words.
- (20 pts) Design and implementation of the MEMM, as well as the feature set and how the feature extraction is done.
- (20 pts) Experiment design and methodology. Investigations into feature sets and unknown word handling/smoothing.
- (20 pts) Error analysis and comparison of **HMM** model with the **MEMM**. 
- (10 pts) Report: organization, clarity and including the **corresponding pieces of code for the implementations**.
- (5 pts) **Submission to Kaggle**. In the report, you should add a screenshot of your team’s performance on kaggle leaderboard.

